In [ ]:
import json
import tensorflow as tf

# 函数用来parse SQuAD 2.0的数据
# 我们只需要抽取文本信息, SQuAD2.0的数据格式比较奇怪
def read_squad_examples(input_file, is_training):
    with tf.io.gfile.GFile(input_file, "r") as reader:
        input_data = json.load(reader)["data"]
    return input_data
    
json_data = read_squad_examples("./input_data/train-v2.0.json", False)

len(json_data)

In [ ]:
json_data[1]